# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from ipywidgets import VBox, HBox, Dropdown, ToggleButtons, Output
from IPython.display import display

In [2]:
# 1. BRFSS 2015 (from Kaggle)
df_brfss = pd.read_csv("diabetes_binary_5050split_health_indicators_BRFSS2015.csv")

# standardising column names 
df_brfss.columns = df_brfss.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-','_')

In [3]:
df_brfss.head()

,diabetes_binary,highbp,highchol,cholcheck,bmi,smoker,stroke,heartdiseaseorattack,physactivity,fruits,...,anyhealthcare,nodocbccost,genhlth,menthlth,physhlth,diffwalk,sex,age,education,income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [4]:
df_brfss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   diabetes_binary       70692 non-null  float64
 1   highbp                70692 non-null  float64
 2   highchol              70692 non-null  float64
 3   cholcheck             70692 non-null  float64
 4   bmi                   70692 non-null  float64
 5   smoker                70692 non-null  float64
 6   stroke                70692 non-null  float64
 7   heartdiseaseorattack  70692 non-null  float64
 8   physactivity          70692 non-null  float64
 9   fruits                70692 non-null  float64
 10  veggies               70692 non-null  float64
 11  hvyalcoholconsump     70692 non-null  float64
 12  anyhealthcare         70692 non-null  float64
 13  nodocbccost           70692 non-null  float64
 14  genhlth               70692 non-null  float64
 15  menthlth           

In [5]:
# distribution of classes in dataset
print(df_brfss['diabetes_binary'].value_counts())
print(df_brfss['diabetes_binary'].value_counts(normalize=True))

diabetes_binary
0.0    35346
1.0    35346
Name: count, dtype: int64
diabetes_binary
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64


# Hypothesis 1: Lifestyle Habits and Diabetes

### Individual Lifestyle Factors vs. Diabetes 


In [45]:
FACTOR_ORDER = [
    ("smoker",            "Smoking"),
    ("physactivity",      "No Physical Activity"),
    ("fruits",            "Low Fruit Intake"),
    ("veggies",           "Low Veggie Intake"),
    ("hvyalcoholconsump", "Heavy Alcohol Consumption"),
]
RISK_VALUE = {"smoker":1, "physactivity":0, "fruits":0, "veggies":0, "hvyalcoholconsump":1}
COLORS = {"With Risk": "#A64A47", "Without Risk": "#E8C6AE"}  

need = {k for k,_ in FACTOR_ORDER} | {"diabetes_binary"}
missing = need - set(df_brfss.columns)
assert not missing, f"Missing columns: {missing}"

df_long = df_brfss.melt(
    id_vars="diabetes_binary",
    value_vars=[k for k,_ in FACTOR_ORDER],
    var_name="factor",
    value_name="value"
)

df_long["risk_val"] = df_long["factor"].map(RISK_VALUE)
df_long["group"]    = np.where(df_long["value"] == df_long["risk_val"], "With Risk", "Without Risk")
label_map = dict(FACTOR_ORDER)
df_long["Factor"] = df_long["factor"].map(label_map)

g = df_long.groupby(["Factor","group"])["diabetes_binary"]
summary = g.agg(sum="sum", n="count", prev="mean").reset_index()

cat_order = [lbl for _, lbl in FACTOR_ORDER]
summary["Factor"] = pd.Categorical(summary["Factor"], categories=cat_order, ordered=True)
summary = summary.sort_values(["Factor","group"])

ymax = max(0.05, float(summary["prev"].max()) * 1.25)
ymax = min(1.0, ymax)  

fig = go.Figure()

for i, y0 in enumerate(np.arange(0, ymax, 0.10)):
    if i % 2 == 0:
        fig.add_hrect(y0=y0, y1=min(y0+0.10, ymax),
                      fillcolor="#F2F4F7", line_width=0, layer="below")

buttons = []
total_traces = 2 * len(cat_order)

for fi, f_label in enumerate(cat_order):
    sub = summary[summary["Factor"] == f_label]
    for group_name in ["With Risk", "Without Risk"]:
        s = sub[sub["group"] == group_name]
        fig.add_bar(
            name=group_name,
            legendgroup=group_name,
            showlegend=(fi == 0),               
            x=s["Factor"],                       
            y=s["prev"],
            text=(s["prev"]*100).round(1).astype(str) + "%",
            textposition="outside",
            marker=dict(color=COLORS[group_name],
                        line=dict(color="rgba(0,0,0,0.12)", width=1)),
            customdata=np.stack([s["n"], s["sum"]], axis=-1),
            hovertemplate=(
                "Factor: %{x}<br>%{fullData.name}: %{y:.1%}"
                "<br>n=%{customdata[0]}  cases=%{customdata[1]}"
                "<extra></extra>"
            ),
            offsetgroup=group_name
        )

initial_vis = [False] * total_traces
initial_vis[0] = True; initial_vis[1] = True  
for i, vis in enumerate(initial_vis):
    fig.data[i].visible = vis

for fi, f_label in enumerate(cat_order):
    vis = [False]*total_traces
    vis[2*fi] = True; vis[2*fi+1] = True
    buttons.append(dict(
        label=f_label,
        method="update",
        args=[{"visible": vis},
              {"xaxis": {"title": "",
                         "categoryorder": "array",
                         "categoryarray": [f_label]}}]
    ))

fig.update_traces(cliponaxis=False, textfont_size=11)
fig.update_layout(
    title=dict(text="Individual Lifestyle Factors<br><sup>Diabetes rate with vs without risk</sup>",
               x=0.02, xanchor="left"),
    barmode="group", bargap=0.28, bargroupgap=0.14,
    xaxis=dict(title="Lifestyle Factors", showgrid=False),
    yaxis=dict(title="Diabetes Rate (%)", tickformat=".0%", dtick=0.10, range=[0, ymax], showgrid=False),
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.18),
    template="simple_white",
    margin=dict(l=70, r=30, t=80, b=90),
    updatemenus=[dict(
        type="dropdown", x=1.0, xanchor="right", y=1.12, yanchor="top",
        buttons=buttons, direction="down", pad={"r": 6, "t": 6}
    )]
)

fig.show()


In [55]:
need = {k for k,_ in FACTOR_ORDER} | {"diabetes_binary"}
missing = need - set(df_brfss.columns)
assert not missing, f"Missing columns: {missing}"

df_long = df_brfss.melt(
    id_vars="diabetes_binary",
    value_vars=[k for k,_ in FACTOR_ORDER],
    var_name="factor",
    value_name="value"
)
risk_map = pd.Series(RISK_VALUE)
df_long["group"] = np.where(df_long["value"] == df_long["factor"].map(risk_map),
                            "With Risk", "Without Risk")
label_map = dict(FACTOR_ORDER)
df_long["Factor"] = df_long["factor"].map(label_map)

summary = (df_long
           .groupby(["Factor","group"])["diabetes_binary"]
           .agg(sum="sum", n="count", prev="mean")
           .reset_index())

cat_order = [lbl for _, lbl in FACTOR_ORDER]
summary["Factor"] = pd.Categorical(summary["Factor"], categories=cat_order, ordered=True)
summary = summary.sort_values(["Factor","group"])

# plot
ymax = max(0.05, float(summary["prev"].max()) * 1.25)
ymax = min(1.0, ymax)  # cap at 100%

fig = go.Figure()

for i, y0 in enumerate(np.arange(0, ymax, 0.10)):
    if i % 2 == 0:
        fig.add_hrect(y0=y0, y1=min(y0+0.10, ymax),
                      fillcolor="#F2F4F7", line_width=0, layer="below")

hover = (
    "Factor: %{x}<br>%{fullData.name}: %{y:.1%}"
    "<br>n=%{customdata[0]}  cases=%{customdata[1]}<extra></extra>"
)

for name, color in [("With Risk","#A64A47"), ("Without Risk","#E8C6AE")]:
    s = summary[summary["group"] == name]
    fig.add_bar(
        name=name, legendgroup=name,
        x=s["Factor"], y=s["prev"],
        text=(s["prev"]*100).round(1).astype(str) + "%",
        textposition="outside",
        marker=dict(color=color, line=dict(color="rgba(0,0,0,0.12)", width=1)),
        customdata=np.stack([s["n"], s["sum"]], axis=-1),
        hovertemplate=hover,
        offsetgroup=name
    )

fig.update_traces(cliponaxis=False, textfont_size=11)
fig.update_layout(
    title=dict(text="Individual Lifestyle Factors<br><sup>Diabetes rate with vs without risk</sup>",
               x=0.02, xanchor="left"),
    barmode="group", bargap=0.28, bargroupgap=0.14,
    xaxis=dict(title="Lifestyle Factors", showgrid=False),
    yaxis=dict(title="Diabetes Rate (%)", tickformat=".0%", dtick=0.10, range=[0, ymax], showgrid=False),
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.18),
    template="simple_white",
    margin=dict(l=70, r=30, t=80, b=90)
)

fig.show()


In [56]:
RISK_VALUE = {"smoker":1, "physactivity":0, "fruits":0, "veggies":0, "hvyalcoholconsump":1}
COLOR_MAP  = {"With Risk":"#A64A47", "Without Risk":"#E8C6AE"}
AGE_ORDER  = ["18–29","30–44","45–59","60–74","75+"]
CAT_ORDER  = [lbl for _, lbl in FACTOR_ORDER]

d = df_brfss.copy()
d.columns = d.columns.str.lower()

db = pd.to_numeric(d.get("diabetes_binary"), errors="coerce")
d = d.assign(diabetes_binary=(db == 1).astype(float)).dropna(subset=["diabetes_binary"])

if "age" in d.columns:
    age = pd.to_numeric(d["age"], errors="coerce")
    if age.dropna().between(1,13).all():
        def _map_code(a):
            if a in (1,2): return "18–29"
            if a in (3,4,5): return "30–44"
            if a in (6,7,8): return "45–59"
            if a in (9,10,11): return "60–74"
            if a in (12,13): return "75+"
            return np.nan
        d["agegroup"] = age.astype("Int64").map(_map_code)
    else:
        d["agegroup"] = pd.cut(age, [18,30,45,60,75,np.inf],
                               labels=AGE_ORDER, right=False, include_lowest=True)

fcols = [k for k,_ in FACTOR_ORDER]
idvars = ["diabetes_binary"] + (["agegroup"] if "agegroup" in d.columns else [])
df_long = d.melt(id_vars=idvars, value_vars=fcols,
                 var_name="factor", value_name="value")
df_long["value"] = pd.to_numeric(df_long["value"], errors="coerce").where(lambda s: s.isin([0,1]))
df_long = df_long.dropna(subset=["value"])

risk_map = pd.Series(RISK_VALUE)
df_long["group"] = np.where(df_long["value"] == df_long["factor"].map(risk_map),
                            "With Risk", "Without Risk")
df_long["Factor"] = pd.Categorical(df_long["factor"].map(dict(FACTOR_ORDER)),
                                   categories=CAT_ORDER, ordered=True)

gkeys = ["Factor","group"] + (["agegroup"] if "agegroup" in df_long.columns else [])
summary = (df_long.groupby(gkeys, observed=True)["diabetes_binary"]
                  .agg(sum="sum", n="count", prev="mean")
                  .reset_index())
summary["text"] = (summary["prev"]*100).round(1).astype(str) + "%"

ymax = min(1.0, max(0.05, float(summary["prev"].max())*1.25))

fig = px.bar(
    summary, x="Factor", y="prev", color="group",
    barmode="group", text="text",
    animation_frame=("agegroup" if "agegroup" in summary.columns else None),
    color_discrete_map=COLOR_MAP,
    category_orders={"Factor": CAT_ORDER, "agegroup": AGE_ORDER},
    title="Diabetes Rate by Lifestyle Behaviour" + ("" if "agegroup" not in summary.columns else " — by Age Group")
)

fig.update_traces(
    textposition="outside",
    hovertemplate="<b>%{x}</b><br>%{legendgroup}: %{y:.1%}<br>n=%{customdata[0]}  cases=%{customdata[1]}<extra></extra>",
    customdata=np.c_[summary["n"], summary["sum"]],
    marker_line_width=1, marker_line_color="rgba(0,0,0,0.12)"
)

for i, y0 in enumerate(np.arange(0, ymax, 0.10)):
    if i % 2 == 0:
        fig.add_hrect(y0=y0, y1=min(y0+0.10, ymax),
                      fillcolor="#F2F4F7", line_width=0, layer="below")

fig.update_layout(
    template="simple_white",
    bargap=0.28, bargroupgap=0.14, showlegend=True,
    xaxis=dict(title="", showgrid=False, categoryorder="array", categoryarray=CAT_ORDER),
    yaxis=dict(title="Diabetes Rate (%)", tickformat=".0%", dtick=0.10, range=[0, ymax], showgrid=False),
    margin=dict(l=70, r=30, t=80, b=80)
)

fig.show()


### Diabetes prevalence increases with multiple lifestyle risk factors
*Count lifestyle risks per person (smoking, inactivity, low fruit/veg, heavy alcohol), compute diabetes prevalence for 0,1,2,3,4+ factors, and plot a clean bar chart (with soft bands) to show the increase in prevalence as risks accumulate.*

In [57]:
df = df_brfss.copy()
cols = {
    "smoke":    ("smoker",            1),
    "inactive": ("physactivity",      0),
    "lowfruit": ("fruits",            0),
    "lowveg":   ("veggies",           0),
    "heavyalc": ("hvyalcoholconsump", 1),
}

rb = pd.DataFrame(index=df.index)
for new, (col, risky) in cols.items():
    if col in df.columns:
        rb[new] = (pd.to_numeric(df[col], errors="coerce") == risky).astype(int)

if rb.shape[1] == 0:
    rb["none"] = 0  

df["risk_behaviors"] = rb.sum(axis=1).fillna(0).astype(int)

order = ["0 factors","1 factor","2 factors","3 factors","4+ factors"]
labels = lambda k: "4+ factors" if k >= 4 else f"{k} factor{'s' if k != 1 else ''}"

prev_rb = (df.assign(diabetes_binary=(pd.to_numeric(df["diabetes_binary"], errors="coerce") == 1).astype(float))
             .groupby("risk_behaviors", as_index=False)["diabetes_binary"].mean()
             .rename(columns={"diabetes_binary":"prevalence"}))

prev_rb["group"] = prev_rb["risk_behaviors"].map(labels)
prev_rb = (prev_rb.groupby("group", as_index=False)["prevalence"].mean())
prev_rb["group"] = pd.Categorical(prev_rb["group"], categories=order, ordered=True)
prev_rb = prev_rb.sort_values("group")

# Plot
ymax = min(1.0, max(0.05, float(prev_rb["prevalence"].max()) * 1.25))
step = 0.10

fig = go.Figure(go.Bar(
    x=prev_rb["group"].astype(str),
    y=prev_rb["prevalence"],
    text=(prev_rb["prevalence"]*100).round(1).astype(str) + "%",
    textposition="outside",
    marker=dict(color="#a64a47", line=dict(width=1, color="rgba(0,0,0,0.15)")),
    hovertemplate="<b>%{x}</b><br>Diabetes rate: %{y:.1%}<extra></extra>",
    cliponaxis=False
))

for i, y0 in enumerate(np.arange(0, ymax, step)):
    if i % 2 == 0:
        fig.add_hrect(y0=y0, y1=min(y0+step, ymax),
                      fillcolor="#F2F4F7", line_width=0, layer="below")

fig.update_layout(
    title=dict(text="Diabetes prevalence increases with multiple lifestyle risk factors", x=0.02),
    template="simple_white",
    bargap=0.35, showlegend=False,
    margin=dict(l=60, r=20, t=60, b=60),
    xaxis=dict(title="Number of factors", categoryorder="array",
               categoryarray=order, showgrid=False),
    yaxis=dict(title="Diabetes Rate (%)", tickformat=".0%", dtick=0.10,
               range=[0, ymax], showgrid=False),
    uniformtext_minsize=10, uniformtext_mode="hide",
    paper_bgcolor="white", plot_bgcolor="white"
)

fig.show()


### Relative Risk Radar
*Compute the % change in diabetes prevalence for each factor and visualize it on a radar chart.*

In [58]:
FACTOR_ORDER = [
    ("smoker",            "Smoking"),
    ("physactivity",      "No Physical Activity"),
    ("fruits",            "Low Fruit Intake"),
    ("veggies",           "Low Veggie Intake"),
    ("hvyalcoholconsump", "Heavy Alcohol Consumption"),
]
RISK_VALUE = {"smoker":1, "physactivity":0, "fruits":0, "veggies":0, "hvyalcoholconsump":1}
CAT_ORDER  = [label for _, label in FACTOR_ORDER]

df = df_brfss.copy()
df.columns = df.columns.str.lower()
df["diab01"] = (pd.to_numeric(df["diabetes_binary"], errors="coerce") == 1).astype(float)

fcols = [k for k,_ in FACTOR_ORDER if k in df.columns]
for c in fcols:
    df[c] = pd.to_numeric(df[c], errors="coerce").where(lambda s: s.isin([0,1]))

long = df.melt(id_vars="diab01", value_vars=fcols, var_name="factor", value_name="value").dropna()
long["group"]  = np.where(long["value"] == long["factor"].map(RISK_VALUE), "With Risk", "Without Risk")
long["Factor"] = pd.Categorical(long["factor"].map(dict(FACTOR_ORDER)), categories=CAT_ORDER, ordered=True)

df_bars = (long.groupby(["Factor","group"], observed=True)["diab01"]
                .mean()
                .unstack("group")
                .reindex(CAT_ORDER)
                .reset_index())

wr = df_bars["With Risk"].astype(float).to_numpy()
wo = df_bars["Without Risk"].astype(float).to_numpy()
ratio = np.divide(wr, wo, out=np.full_like(wr, np.nan), where=wo > 0)
df_bars["% Increase"] = (ratio - 1) * 100

factors = df_bars["Factor"].astype(str).to_numpy()
delta   = df_bars["% Increase"].astype(float).to_numpy()

theta = np.r_[factors, factors[:1]] if len(factors) else factors
r     = np.r_[delta,   delta[:1]]   if len(delta)   else delta

vmax = np.nanmax(delta) if np.isfinite(delta).any() else 0.0
vmin = np.nanmin(delta) if np.isfinite(delta).any() else 0.0
rmax = max(10.0, 10.0 * np.ceil((vmax*1.10)/10.0))
rmin = 10.0 * np.floor((min(0.0, vmin*1.10))/10.0)

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    theta=theta, r=r,
    mode="lines",
    line=dict(color="#a64a47", width=3, shape="spline"),
    fill="toself", fillcolor="rgba(166,74,71,0.22)",
    hoverinfo="text",
    text=[f"{lab}: {val:+.1f}%" for lab, val in zip(theta, r)],
    hovertext=[f"<b>{lab}</b><br>Δ risk: {val:+.1f}%" for lab, val in zip(theta, r)],
    showlegend=False
))

fig.add_trace(go.Scatterpolar(
    theta=factors, r=delta,
    mode="markers+text",
    marker=dict(size=8, color="#a64a47", line=dict(width=1, color="rgba(0,0,0,0.2)")),
    text=[f"{v:+.0f}%" if np.isfinite(v) else "" for v in delta],
    textposition="top center",
    hovertemplate="<b>%{theta}</b><br>Δ risk: %{r:+.1f}%<extra></extra>",
    showlegend=False
))

fig.update_layout(
    title=dict(text="Relative Risk Comparison<br><sup>% change in diabetes rate by factor</sup>", x=0.03, xanchor="left"),
    template="simple_white",
    polar=dict(
        bgcolor="white",
        angularaxis=dict(rotation=90, direction="clockwise"),
        radialaxis=dict(range=[rmin, rmax], tick0=0, dtick=10, ticksuffix="%")
    ),
    margin=dict(l=40, r=40, t=80, b=40)
)

fig.show()

### Physical Activity vs Diabetes by Education, AgeGroup and Sex

In [59]:
EDU_MAP = {
    1: "K-only / None",
    2: "Grades 1–8",
    3: "Grades 9–11",
    4: "HS Grad / GED",
    5: "Some College / AA",
    6: "College 4+",
}
EDU_ORDER = [EDU_MAP[k] for k in (1,2,3,4,5,6)]
AGE_ORDER = ["18–29","30–44","45–59","60–74","75+"]

COLOR_NO  = "#a64a47"   # inactive = No
COLOR_YES = "#D6B4B3"   # active   = Yes
GRID      = "#e9eef3"

def normalize(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()
    df.columns = df.columns.str.lower()

    # diabetes -> 0/1
    db = pd.to_numeric(df.get("diabetes_binary"), errors="coerce")
    df["diabetes"] = (db == 1).astype(float)

    # physactivity -> Yes/No
    pa = pd.to_numeric(df.get("physactivity"), errors="coerce")
    pa = pa.where(pa.isin([0,1]), np.where(pa == 1, 1, 0))  
    df["phys_lbl"] = np.where(pa == 1, "Yes", "No")

    age = pd.to_numeric(df.get("age"), errors="coerce")
    if age.dropna().between(1,13).all():
        def map_agecode(a):
            if a in (1,2): return "18–29"
            if a in (3,4,5): return "30–44"
            if a in (6,7,8): return "45–59"
            if a in (9,10,11): return "60–74"
            if a in (12,13): return "75+"
            return np.nan
        df["agegroup"] = age.astype("Int64").map(map_agecode)
    else:
        df["agegroup"] = pd.cut(age, [18,30,45,60,75,np.inf],
                                labels=AGE_ORDER, right=False, include_lowest=True)

    edu = pd.to_numeric(df.get("education"), errors="coerce").astype("Int64")
    df["education_lbl"] = pd.Categorical(edu.map(EDU_MAP),
                                         categories=EDU_ORDER, ordered=True)

    sex_raw = df.get("sex")
    if sex_raw is not None:
        s = pd.to_numeric(sex_raw, errors="coerce")
        if s.dropna().isin([0,1]).all():
            df["sex_lbl"] = s.map({0:"Female", 1:"Male"})
        elif s.dropna().isin([1,2]).all():
            df["sex_lbl"] = s.map({1:"Male", 2:"Female"})
        else:
            v = sex_raw.astype(str).str.lower().str.strip()
            df["sex_lbl"] = np.where(v.isin(["f","female"]), "Female",
                              np.where(v.isin(["m","male"]), "Male", np.nan))
    else:
        df["sex_lbl"] = np.nan

    return df

def make_table(df: pd.DataFrame, facet_var: str) -> pd.DataFrame:
    sub = df.dropna(subset=["diabetes","phys_lbl", facet_var]).copy()
    g = (sub.groupby([facet_var, "phys_lbl"], observed=True)
           .agg(n=("diabetes","size"), cases=("diabetes","sum"))
           .reset_index())
    g["prev"] = g["cases"] / g["n"]
    g["phys_lbl"] = pd.Categorical(g["phys_lbl"], ["No","Yes"], ordered=True)
    if facet_var == "agegroup":
        g["agegroup"] = pd.Categorical(g["agegroup"], AGE_ORDER, ordered=True)
    if facet_var == "education_lbl":
        g["education_lbl"] = pd.Categorical(g["education_lbl"], EDU_ORDER, ordered=True)
    return g.sort_values([facet_var, "phys_lbl"]).reset_index(drop=True)

def facet_plot(tbl: pd.DataFrame, facet_var: str, title_note: str):
    category_orders = {"phys_lbl":["No","Yes"]}
    if facet_var == "agegroup":       category_orders["agegroup"] = AGE_ORDER
    if facet_var == "education_lbl":  category_orders["education_lbl"] = EDU_ORDER

    fig = px.bar(
        tbl, x="phys_lbl", y="prev",
        facet_col=facet_var, facet_col_wrap=3,
        color="phys_lbl",
        category_orders=category_orders,
        color_discrete_map={"No": COLOR_NO, "Yes": COLOR_YES},
        text=tbl["prev"].map(lambda v: f"{v:.0%}"),
        title=f"Physical Activity vs Diabetes by {title_note}<br><sup>% with diabetes — faceted</sup>"
    )
    fig.update_traces(
        marker_line_width=1, marker_line_color="rgba(0,0,0,0.15)",
        textposition="outside",
        hovertemplate="%{x}<br>%{y:.1%}<extra></extra>"
    )
    fig.for_each_yaxis(lambda a: a.update(title="% Diabetes", tickformat=".0%", dtick=0.10, range=[0,1], gridcolor=GRID))
    fig.for_each_xaxis(lambda a: a.update(title="Physically Active"))
    fig.update_layout(template="simple_white", bargap=0.35, showlegend=False,
                      margin=dict(l=50, r=20, t=70, b=50))
    return fig

def animate_by_age(tbl_age: pd.DataFrame):
    frames = []
    for ag in AGE_ORDER:
        t = tbl_age[tbl_age["agegroup"] == ag]
        if t.empty: 
            continue
        frames.append(go.Frame(
            name=str(ag),
            data=[go.Bar(
                x=t["phys_lbl"], y=t["prev"],
                text=t["prev"].map(lambda v: f"{v:.0%}"),
                marker_line_width=1, marker_line_color="rgba(0,0,0,0.15)",
                marker_color=[COLOR_NO if v=="No" else COLOR_YES for v in t["phys_lbl"]],
                showlegend=False,
                hovertemplate="%{x}<br>%{y:.1%}<extra></extra>"
            )]
        ))

    fig = go.Figure(data=(frames[0].data if frames else []), frames=frames)
    fig.update_layout(
        title="Physical Activity vs Diabetes — Animated by Age Group",
        template="simple_white",
        xaxis=dict(title="Physically Active", categoryorder="array", categoryarray=["No","Yes"]),
        yaxis=dict(title="% Diabetes", tickformat=".0%", dtick=0.10, range=[0,1], gridcolor=GRID),
        updatemenus=[dict(type="buttons", showactive=False, x=1, y=1.12, xanchor="right",
                          buttons=[
                              dict(label="▶ Play",  method="animate", args=[None, {"fromcurrent": True, "frame": {"duration": 900, "redraw": True}, "transition": {"duration": 200}}]),
                              dict(label="⏸ Pause", method="animate", args=[[None], {"mode":"immediate", "frame":{"duration":0,"redraw":False}, "transition":{"duration":0}}])
                          ])],
        sliders=[dict(
            active=0, x=0.05, y=1.05, xanchor="left", len=0.7,
            currentvalue=dict(prefix="Age Group: ", visible=True),
            steps=[dict(method="animate", label=f.name, args=[[f.name], {"mode":"immediate","frame":{"duration":0,"redraw":True}, "transition":{"duration":0}}]) for f in frames]
        )],
        margin=dict(l=50, r=20, t=90, b=50)
    )
    return fig

def show_physactivity(df_brfss: pd.DataFrame):
    df = normalize(df_brfss)

    facet_dd = Dropdown(
        options=[("Education","education_lbl"), ("Age Group","agegroup"), ("Sex","sex_lbl")],
        value="education_lbl", description="Facet:"
    )
    view_toggle = ToggleButtons(
        options=[("Faceted","facet"), ("Animate by Age","animate")],
        value="facet", description="View:"
    )
    out = Output()

    def render(*_):
        out.clear_output(wait=True)
        with out:
            if view_toggle.value == "facet":
                tbl = make_table(df, facet_dd.value)
                pretty = dict(education_lbl="Education", agegroup="Age Group", sex_lbl="Sex")[facet_dd.value]
                fig = facet_plot(tbl, facet_var=facet_dd.value, title_note=pretty)
                fig.show()
            else:
                tbl_age = make_table(df, "agegroup")
                fig = animate_by_age(tbl_age)
                fig.show()

    facet_dd.observe(render, names="value")
    view_toggle.observe(render, names="value")

    render()
    display(VBox([HBox([facet_dd, view_toggle]), out]))

show_physactivity(df_brfss)


# Hypothesis 2 — Education & Diabetes Prevention

### Education & Health Behaviors
*Recode education into four levels, compute the % with Healthy Diet, Physical Activity, and Regular Checkups by education group, and visualize them as grouped bars to show healthier behaviors with higher education.*

In [60]:
df = df_brfss.copy()
df.columns = df.columns.str.lower()

edu_col = "educa" if "educa" in df.columns else "education"
map4 = {1:"Less than HS", 2:"Less than HS", 3:"Less than HS",
        4:"HS Graduate", 5:"Some College", 6:"College Grad"}
order4 = ["Less than HS","HS Graduate","Some College","College Grad"]

edu_num = pd.to_numeric(df.get(edu_col), errors="coerce")
df = df.dropna(subset=[edu_col]).copy()
df["Education"] = pd.Categorical(edu_num.map(map4), categories=order4, ordered=True)

def as01(s):
    s = pd.to_numeric(df.get(s), errors="coerce")
    return s.where(s.isin([0,1])).fillna(0).astype(int)

if {"fruits","veggies"}.issubset(df.columns):
    fruits  = as01("fruits")
    veggies = as01("veggies")
    df["Healthy Diet"] = ((fruits==1) & (veggies==1)).astype(int)
elif "fruits" in df.columns:
    df["Healthy Diet"] = as01("fruits")
else:
    df["Healthy Diet"] = as01("veggies") if "veggies" in df.columns else 0

df["Physical Activity"] = as01("physactivity") if "physactivity" in df.columns else 0

if "cholcheck" in df.columns:
    df["Regular Checkups"] = as01("cholcheck")
elif "anyhealthcare" in df.columns:
    df["Regular Checkups"] = as01("anyhealthcare")
else:
    df["Regular Checkups"] = 0

agg = (df.groupby("Education", observed=True)[
        ["Healthy Diet","Physical Activity","Regular Checkups"]
      ].mean().mul(100).reset_index())

long = agg.melt(id_vars="Education", var_name="Metric", value_name="Prevalence")

# Plot 
colors = {
    "Healthy Diet":      "#8C1D18",  
    "Physical Activity": "#C94B44",  
    "Regular Checkups":  "#ECD9C6",  
}

fig = px.bar(
    long, x="Education", y="Prevalence", color="Metric",
    barmode="group", text="Prevalence",
    color_discrete_map=colors,
    title="Higher education predicts better health behaviors across domains",
    labels={"Prevalence":"Prevalence (%)"},
    category_orders={"Education": order4,
                     "Metric": ["Healthy Diet","Physical Activity","Regular Checkups"]},
    template="simple_white"
)

fig.update_layout(shapes=[
    dict(type="rect", xref="paper", x0=0, x1=1, y0=y, y1=min(y+20,100),
         layer="below", line=dict(width=0), fillcolor="#F2F4F7")
    for i, y in enumerate(np.arange(0, 100, 20)) if i % 2 == 0
])

fig.update_traces(
    texttemplate="%{text:.0f}%",
    textposition="outside",
    cliponaxis=False,
    hovertemplate="<b>%{x}</b><br>%{fullData.name}: %{y:.0f}%<extra></extra>"
)

fig.update_layout(
    yaxis=dict(range=[0,100], dtick=20, title="Prevalence (%)", showgrid=False),
    xaxis=dict(title="Education Level", showgrid=False),
    legend=dict(orientation="h", x=0.5, xanchor="center", y=-0.22),
    bargap=0.28, bargroupgap=0.16,
    margin=dict(l=70, r=40, t=80, b=100),
    uniformtext_minsize=10, uniformtext_mode="hide",
    paper_bgcolor="white", plot_bgcolor="white"
)

fig.show()


### Education & Diabetes Trend 
*The chart shows diabetes rates decline as education increases across the four groups.*

In [61]:
df = df_brfss.dropna(subset=["diabetes_binary"]).copy()
df.columns = df.columns.str.lower()

edu_col = "educa" if "educa" in df.columns else "education"
map4   = {1:"Less than HS", 2:"Less than HS", 3:"Less than HS",
          4:"HS Graduate",  5:"Some College", 6:"College Grad"}
order4 = ["Less than HS","HS Graduate","Some College","College Grad"]

edu_num = pd.to_numeric(df.get(edu_col), errors="coerce")
df = df.dropna(subset=[edu_col]).copy()
df["edu4"] = pd.Categorical(edu_num.map(map4), categories=order4, ordered=True)

diab = (pd.to_numeric(df["diabetes_binary"], errors="coerce") == 1).astype(int)
g = df.assign(d=diab).groupby("edu4", observed=True)["d"]
tab = (g.agg(n="count", cases="sum", prev="mean")
         .reset_index()
         .rename(columns={"edu4":"Education"}))
tab["rate"] = (tab["prev"]*100)

# plot 
x  = tab["Education"].astype(str)
y  = tab["rate"]

fig = go.Figure()

ymax = min(100.0, max(5.0, float(y.max())*1.25))
for i, y0 in enumerate(np.arange(0, ymax, 10)):
    if i % 2 == 0:
        fig.add_hrect(y0=y0, y1=min(y0+10, ymax),
                      fillcolor="#F2F4F7", line_width=0, layer="below")

fig.add_trace(go.Scatter(
    x=x, y=y, mode="lines+markers+text",
    line=dict(color="#A64A47", width=3),
    marker=dict(size=9, color="white", line=dict(color="#A64A47", width=2)),
    text=[f"{v:.1f}%" for v in y],
    textposition="top center",
    hovertemplate="%{x}<br>Diabetes Rate: %{y:.1f}%<br>n=%{customdata[0]}  cases=%{customdata[1]}<extra></extra>",
    customdata=np.c_[tab["n"], tab["cases"]],
    name="Diabetes rate"
))

fig.update_layout(
    title="Diabetes Rate Declines with Higher Education (4 groups)",
    template="simple_white",
    margin=dict(l=70, r=30, t=80, b=70),
    xaxis=dict(title="Education Level", showgrid=False),
    yaxis=dict(title="Diabetes Rate (%)", range=[0, ymax], dtick=10, showgrid=False),
    showlegend=False
)

fig.show()

### Income vs Diabetes by Education Group 


In [62]:
INCOME_MAP = {1:"< $10k", 2:"$10–15k", 3:"$15–20k", 4:"$20–25k",
              5:"$25–35k", 6:"$35–50k", 7:"$50–75k", 8:"≥ $75k"}
INC_ORDER = [INCOME_MAP[k] for k in (1,2,3,4,5,6,7,8)]

AGE_ORDER  = ["18–29","30–44","45–59","60–74","75+"]
RED_PALETTE = ["#fde0dd", "#fcbba1", "#fc9272", "#fb6a4a", "#ef3b2c", "#cb181d"]

def _clean_ei(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()
    df.columns = df.columns.str.lower()

    db = pd.to_numeric(df.get("diabetes_binary"), errors="coerce")
    df["diabetes"] = (db == 1).astype(int)

    edu = pd.to_numeric(df.get("education"), errors="coerce").astype("Int64")
    df["education_lbl"] = pd.Categorical(edu.map(EDU_MAP), categories=EDU_ORDER, ordered=True)

    inc = pd.to_numeric(df.get("income"), errors="coerce").astype("Int64")
    df["income_lbl"] = pd.Categorical(inc.map(INCOME_MAP), categories=INC_ORDER, ordered=True)

    sex_raw = df.get("sex")
    if sex_raw is not None:
        s = pd.to_numeric(sex_raw, errors="coerce")
        if s.dropna().isin([0,1]).all():
            df["sex_lbl"] = s.map({0:"Female", 1:"Male"})
        elif s.dropna().isin([1,2]).all():
            df["sex_lbl"] = s.map({1:"Male", 2:"Female"})
        else:
            v = sex_raw.astype(str).str.lower().str.strip()
            df["sex_lbl"] = np.where(v.isin(["f","female"]), "Female",
                              np.where(v.isin(["m","male"]), "Male", np.nan))
    else:
        df["sex_lbl"] = np.nan

    age = pd.to_numeric(df.get("age"), errors="coerce")
    if age.dropna().between(1,13).all():
        def _five(a):
            if a in (1,2):   return "18–29"
            if a in (3,4,5): return "30–44"
            if a in (6,7,8): return "45–59"
            if a in (9,10,11): return "60–74"
            if a in (12,13):  return "75+"
            return np.nan
        df["agegroup"] = age.astype("Int64").map(_five)
    else:
        df["agegroup"] = pd.cut(age, [18,30,45,60,75,np.inf],
                                labels=AGE_ORDER, right=False, include_lowest=True)

    return df.dropna(subset=["diabetes","education_lbl","income_lbl"])

def _aggregate_ei(df: pd.DataFrame) -> pd.DataFrame:
    g = (df.groupby(["education_lbl","income_lbl"], observed=True)
           .agg(n=("diabetes","size"),
                cases=("diabetes","sum"),
                prev=("diabetes","mean"))
           .reset_index())
    g["text"] = (g["prev"]*100).round(0).astype(int).astype(str) + "%"
    return g

# Bars 
def _bars(tbl: pd.DataFrame):
    fig = px.bar(
        tbl, x="income_lbl", y="prev", color="education_lbl",
        category_orders={"income_lbl": INC_ORDER, "education_lbl": EDU_ORDER},
        barmode="group", text="text",
        color_discrete_sequence=RED_PALETTE,
        title="Income vs Diabetes — Grouped by Education"
    )
    fig.update_traces(
        textposition="outside",
        marker_line_width=1, marker_line_color="rgba(0,0,0,0.18)",
        customdata=np.c_[tbl["n"]],
        hovertemplate="<b>%{fullData.name}</b><br>"
                      "Income: %{x}<br>"
                      "Diabetes: %{y:.1%}<br>"
                      "n=%{customdata[0]}<extra></extra>"
    )
    fig.update_layout(
        template="simple_white",
        xaxis_title="Income Scale",
        yaxis=dict(title="% with Diabetes", tickformat=".0%", dtick=0.10, range=[0,1]),
        bargap=0.2, bargroupgap=0.12,
        legend_title="Education",
        margin=dict(l=70, r=20, t=70, b=60)
    )
    return fig

# Heatmap
def _heat(tbl: pd.DataFrame):
    mat = (tbl.pivot(index="education_lbl", columns="income_lbl", values="prev")
               .reindex(index=EDU_ORDER, columns=INC_ORDER))
    n   = (tbl.pivot(index="education_lbl", columns="income_lbl", values="n")
               .reindex(index=EDU_ORDER, columns=INC_ORDER))

    hover = mat.copy().astype(object)
    for i, er in enumerate(EDU_ORDER):
        for j, inc in enumerate(INC_ORDER):
            p = mat.iloc[i,j]; nn = n.iloc[i,j]
            hover.iloc[i,j] = (
                f"<b>{er}</b><br>{inc}<br>"
                + (f"Diabetes: {p:.1%}" if pd.notna(p) else "Diabetes: —")
                + (f"<br>n={int(nn)}" if pd.notna(nn) else "")
            )

    vmax = 1.0 if not np.isfinite(np.nanmax(mat.values)) else max(0.20, float(np.nanmax(mat.values)))
    fig = go.Figure(go.Heatmap(
        z=mat.values, x=INC_ORDER, y=EDU_ORDER,
        colorscale="Reds",
        zmin=0, zmax=vmax,
        colorbar=dict(title="% Diabetes", tickformat=".0%"),
        hoverinfo="text", text=hover.values, hovertemplate="%{text}<extra></extra>"
    ))
    fig.update_layout(
        title="Diabetes Rate Heatmap — Education × Income",
        template="simple_white",
        xaxis=dict(title="Income Scale"),
        yaxis=dict(title="Education Level", autorange="reversed"),
        margin=dict(l=80, r=80, t=70, b=70)
    )
    return fig

def show_income_vs_diabetes_by_education(df_brfss: pd.DataFrame):
    base = _clean_ei(df_brfss)

    view_dd  = ToggleButtons(options=[("Grouped Bars","bars"), ("Heatmap","heat")],
                             value="bars", description="View:")
    slice_dd = Dropdown(options=[("All","__all__"), ("Sex","sex_lbl"), ("Age Group","agegroup")],
                        value="__all__", description="Slice:")
    value_dd = Dropdown(options=[("All","__all__")], value="__all__", description="Value:")
    out = Output()

    def _options(col):
        if col == "__all__":
            return [("All","__all__")]
        vals = base[col].dropna().astype(str)
        order = [v for v in AGE_ORDER if col=="agegroup" and v in set(vals)]
        if not order:
            order = sorted(vals.unique().tolist())
        return [("All","__all__")] + [(v, v) for v in order]

    def _apply_slice(df):
        if slice_dd.value == "__all__" or value_dd.value == "__all__":
            return df
        return df[df[slice_dd.value].astype(str) == str(value_dd.value)]

    def render(*_):
        sub = _apply_slice(base)
        tbl = _aggregate_ei(sub)
        fig = _bars(tbl) if view_dd.value == "bars" else _heat(tbl)
        out.clear_output(wait=True)
        with out: fig.show()

    def on_slice_change(_):
        value_dd.options = _options(slice_dd.value)
        value_dd.value = "__all__"
        render()

    slice_dd.observe(on_slice_change, names="value")
    value_dd.observe(render, names="value")
    view_dd.observe(render, names="value")

    on_slice_change(None)
    display(VBox([HBox([view_dd, slice_dd, value_dd]), out]))

show_income_vs_diabetes_by_education(df_brfss)


### Education vs Lifestyle Factors and Diabetes

In [63]:
# Colors for lines 
COLOR_ACTIVE = "#386cb0"
COLOR_FRUIT  = "#66a61e"
COLOR_VEG    = "#e6ab02"
COLOR_DIAB   = "#a64a47"  # diabetes line

def _normalize(df_raw: pd.DataFrame) -> pd.DataFrame:
    """Clean/label columns; compute helper columns."""
    df = df_raw.copy()
    df.columns = df.columns.str.lower()

    edu = pd.to_numeric(df.get("education", np.nan), errors="coerce").astype("Int64")
    df["education_lbl"] = edu.map(EDU_MAP)
    df["education_lbl"] = pd.Categorical(df["education_lbl"], categories=EDU_ORDER, ordered=True)

    db = pd.to_numeric(df.get("diabetes_binary", np.nan), errors="coerce")
    df["diabetes"] = db.astype(int) if db.dropna().isin([0,1]).all() else (db == 1).astype(int)

    sex_raw = df.get("sex", np.nan)
    sex_num = pd.to_numeric(sex_raw, errors="coerce")
    if sex_num.dropna().isin([0,1]).all():
        df["sex_lbl"] = sex_num.map({0:"Female", 1:"Male"})
    elif sex_num.dropna().isin([1,2]).all():
        df["sex_lbl"] = sex_num.map({1:"Male", 2:"Female"})
    else:
        val = pd.Series(sex_raw, dtype="object").astype(str).str.strip().str.lower()
        df["sex_lbl"] = np.where(val.isin(["f","female"]), "Female",
                          np.where(val.isin(["m","male"]), "Male", "All"))

    def _as01(col):
        s = pd.to_numeric(df.get(col, np.nan), errors="coerce")
        return s.where(s.isin([0,1]), np.nan)

    df["phys_ok"]   = _as01("physactivity")   # 1 = active
    df["fruit_ok"]  = _as01("fruits")         # 1 = eats fruit (not low)
    df["veg_ok"]    = _as01("veggies")        # 1 = eats veg (not low)

    df = df.dropna(subset=["education_lbl", "diabetes"])
    return df

def _aggregate(df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate lifestyle % and diabetes % by Education (and Sex slice if already filtered)."""
    grp = (df.groupby("education_lbl", observed=True)
             .agg(n=("diabetes","size"),
                  diab=("diabetes","mean"),
                  phys_rate=("phys_ok","mean"),
                  fruit_rate=("fruit_ok","mean"),
                  veg_rate=("veg_ok","mean"))
             .reset_index())
    return grp

def _build_multiline(tbl: pd.DataFrame, show_diab_line: bool) -> go.Figure:
    """Create the multi-line chart; hover shows lifestyle % and diabetes %."""
    x = tbl["education_lbl"].astype(str).tolist()

    traces = []
    def add_line(ycol, name, color):
        if ycol not in tbl or tbl[ycol].notna().sum() == 0:
            return
        y = tbl[ycol].values
        diab = tbl["diab"].values
        hover = [
            f"<b>{name}</b><br>Education: {edu}"
            f"<br>{name}: {0 if pd.isna(yv) else yv:.1%}"
            f"<br>Diabetes: {0 if pd.isna(dv) else dv:.1%}"
            for edu, yv, dv in zip(x, y, diab)
        ]
        traces.append(go.Scatter(
            x=x, y=y, mode="lines+markers", name=name,
            line=dict(width=3, color=color),
            marker=dict(size=7, line=dict(width=1, color="rgba(0,0,0,0.25)")),
            hoverinfo="text", hovertext=hover
        ))

    add_line("phys_rate",  "Physically Active", COLOR_ACTIVE)
    add_line("fruit_rate", "Fruit Intake",      COLOR_FRUIT)
    add_line("veg_rate",   "Vegetable Intake",  COLOR_VEG)

    if show_diab_line and ("diab" in tbl):
        traces.append(go.Scatter(
            x=x, y=tbl["diab"].values, mode="lines+markers",
            name="Diabetes Prevalence",
            line=dict(width=3, color=COLOR_DIAB, dash="dash"),
            marker=dict(size=7, line=dict(width=1, color="rgba(0,0,0,0.25)")),
            hovertemplate="<b>Diabetes</b><br>Education: %{x}<br>%{y:.1%}<extra></extra>",
            yaxis="y2"
        ))

    fig = go.Figure(data=traces)
    fig.update_layout(
        title="Education vs Lifestyle Factors and Diabetes",
        template="simple_white",
        xaxis=dict(title="Education Level", categoryorder="array", categoryarray=EDU_ORDER),
        yaxis=dict(title="% Engaged in Healthy Behaviours", tickformat=".0%", dtick=0.10, range=[0,1], gridcolor="#e9eef3"),
        yaxis2=dict(title="% with Diabetes", overlaying="y", side="right", tickformat=".0%", dtick=0.10, range=[0,1]),
        legend_title="Factor",
        margin=dict(l=70, r=70, t=70, b=60),
        hoverlabel_align="left"
    )
    return fig

def show_education_vs_lifestyle(df_brfss: pd.DataFrame):
    df = _normalize(df_brfss)

    sex_opts = [("All","__all__")]
    uniq_sex = [s for s in ["Female","Male"] if (df["sex_lbl"].astype(str) == s).any()]
    sex_opts += [(s, s) for s in uniq_sex]
    sex_dd = Dropdown(options=sex_opts, value="__all__", description="Sex:")

    axis_toggle = ToggleButtons(
        options=[("Lifestyle only","life"), ("Overlay diabetes","dual")],
        value="life", description="Y-axis:"
    )
    out = Output()

    def apply_slice(_df):
        if sex_dd.value == "__all__":
            return _df
        return _df[_df["sex_lbl"].astype(str) == sex_dd.value]

    def render(*_):
        sub = apply_slice(df)
        tbl = _aggregate(sub)
        fig = _build_multiline(tbl, show_diab_line=(axis_toggle.value == "dual"))
        out.clear_output(wait=True)
        with out:
            fig.show()

    sex_dd.observe(render, names="value")
    axis_toggle.observe(render, names="value")

    render()
    display(VBox([HBox([sex_dd, axis_toggle]), out]))

show_education_vs_lifestyle(df_brfss)
